<a href="https://colab.research.google.com/github/Maynex69/Gestion-de-ingresos-hospitalarios/blob/main/MODELO_DE_PREDICCI%C3%93N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
df = pd.read_csv('/content/df.csv')


In [12]:
# Selecciona solo las columnas que deseas conservar
df = df[['Age', 'Medication', 'Medical Condition']]

# Eliminar columnas no deseadas
df = df.drop(columns=[col for col in df.columns if col not in ['Age', 'Medication', 'Medical Condition']])


# Desarrollo del modelo de ML

```
# Esto tiene formato de código
```



In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Aplicar One-Hot Encoding a la columna 'Medical Condition'
df_encoded = pd.get_dummies(df, columns=['Medical Condition'])

# Convertir 'Medication' a códigos numéricos si no es una columna de etiquetas codificadas
df_encoded['Medication'] = df_encoded['Medication'].astype('category').cat.codes

# Separar variables independientes (X) y dependientes (y)
X = df_encoded[['Age'] + [col for col in df_encoded.columns if 'Medical Condition' in col]]
y = df_encoded['Medication']

# Convertir a arrays NumPy
X = np.array(X).astype('float32')
y = np.array(y).astype('int32')  # Convertir y a enteros si es necesario

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir las etiquetas a One-Hot Encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Definir y entrenar el modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))  # Cambiar según el número de clases en y

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1169/1169 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.1827 - loss: 3.0379 - val_accuracy: 0.4207 - val_loss: 1.6053
Epoch 2/50
1169/1169 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.4168 - loss: 1.6007 - val_accuracy: 0.4209 - val_loss: 1.5678
Epoch 3/50
1169/1169 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4179 - loss: 1.5687 - val_accuracy: 0.4159 - val_loss: 1.5594
Epoch 4/50
1169/1169 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.4186 - loss: 1.5566 - val_accuracy: 0.4243 - val_loss: 1.5527
Epoch 5/50
1169/1169 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4166 - loss: 1.5462 - val_accuracy: 0.4186 - val_loss: 1.5521
Epoch 6/50
1169/1169 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4236 - loss: 1.5531 - val_accuracy: 0.4236 - val_loss: 1.5539
Epoch 7/50
1169/1169 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4233 - loss: 1.5401 - val_accuracy: 0.4181 - val_loss: 1.5508
Epoch 8/50
1169/1169 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4224 - loss: 1.5366 - val_accu